In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import stats
import matplotlib as mpl

In [ ]:
# Read Gini index
FeatureImp_df = pd.read_csv("./path/to/GiniIndex.csv",index_col=0)

In [ ]:
# Caliculation of principal variance
kiyo_th = 80
for ct in ["TP-P2","TP-P1","TP-P0","TP-AB","TP-ABa","alpha-P2","alpha-P1","alpha-AB","alpha-ABa","alpha-diff","alpha-diff2"]:
    print("\n"+ct)
    df = new_cat.loc[[i for i in new_cat.index if i.split("_")[0]==ct]]
    df = df.T
    pca = PCA()
    pca.fit(df)
    koyu = pd.DataFrame(pca.explained_variance_, index=["PC{}".format(x+1) for x in range(len(df.index))])
    vec = pd.DataFrame(pca.components_, columns=df.columns, index=["PC{}".format(x + 1) for x in range(len(df.index))])
    vec_sq = vec**2
    pc_th = 0
    for n,i in enumerate(np.cumsum(pca.explained_variance_ratio_)):
        if i > kiyo_th/100:
            pc_th += n+1
            break
    pc_th
    print(pc_th)

    seki = {}
    for g in vec.columns:
        seki[g] = 0
        for pc in koyu.index[:pc_th]:
            seki[g] += vec_sq.loc[pc,g] * koyu.loc[pc,0]
        else:
            seki[g] =  seki[g]/df[g].var()*100
    
    seki_df = pd.DataFrame(seki,index=["seki"]).T.sort_values("seki")
    seki_df.index = [i.split("_")[1] for i in seki_df.index]
    CT_FI = FeatureImp_df.loc[[i for i in FeatureImp_df.index if i.split("_")[0] == ct]]
    CT_FI.index = [i.split("_")[1] for i in CT_FI.index]
    CT_FI+=1
    wa = pd.concat([CT_FI,seki_df],axis=1)
    wa.plot.scatter(x=0,y=1,figsize=(2,2),s=0.5,title=ct,logx=True)#,loglog=True)
    th_sp = wa.corr("spearman").iloc[1,0]
    num_g = 1415
    t_sp = th_sp*np.sqrt((num_g-2)/(1-(th_sp)**2))
    p_sp = stats.t.sf(abs(t_sp), df=num_g-2)
    th_pr = wa.corr("pearson").iloc[1,0]
    t_pr = th_pr*np.sqrt((num_g-2)/(1-(th_pr)**2))
    p_pr = stats.t.sf(abs(t_pr), df=num_g-2)
    wa.to_csv("./path/to/PCA_VARinPC_{}_{}%.csv".format(ct, kiyo_th))
    print(th_sp, str(p_sp), th_pr,  p_pr)

In [ ]:
# Drawing
for ct in ["P0","P1","P2","AB","ABa"]:
    wa = pd.read_csv("./path/to/PCA_VARinPC_{}_{}%.csv".format(ct, 80), index_col=0)
    ky = "TP-"+ct+"_"
    gn_ind = []
    for g in wa.index:
        kyg = ky+g
        gn_ind.append(np.log10(FeatureImp_df.loc[kyg,"Gini"]+0.0005))
        

    wa["Gini"] = gn_ind
    print(wa.corr(method="pearson"))
    fig,ax=plt.subplots(figsize=(1.,1.))
    ax.scatter(wa["Gini"],wa["seki"],s=0.2,color=colors[ct],zorder = 2)

    font = {'family' : 'Arial','size': 7}
    mpl.rc('font', **font)
    ax.xaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
    ax.yaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
    ax.set_axisbelow(True)
    ax.tick_params(labelbottom="on",bottom="on", labelleft="on",left="on", length=1.5,width=1, color=black)
    #ax2.tick_params(labelbottom="on",bottom="on", labelleft="on",left="on", length=1, color=black)
    for j in ["left","bottom"]:
        ax.spines[j].set_color(black)
    for j in ["top","right"]:
        ax.spines[j].set_color("w")
        #ax1.spines[j].set_width(1)
    #plt.xlim(-5.25,-0.75)
    plt.ylim(-5,105)
    th = wa.corr("pearson").iloc[1,0]
    num_g = 1415
    t = th*np.sqrt((num_g-2)/(1-(th)**2))
    p=stats.t.sf(abs(t), df=num_g-2)
    #if ct == "P0":
    #    print(wa.loc["fzy-1"])
    #wa.to_csv("/home/torii/CElegans_LC_TimeSeries/PCA_VARinPC_{}_{}%.csv".format(ct, kiyo_th))
    print(th, str(p), wa.corr("pearson").iloc[1,0])
    print(stats.spearmanr(wa["Gini"].values, wa["seki"].values, alternative='greater'))
    print(stats.spearmanr(wa["Gini"].values, wa["seki"].values, alternative='two-sided'))
    print(stats.pearsonr(wa["Gini"].values, wa["seki"].values, alternative='greater'))
    print(stats.pearsonr(wa["Gini"].values, wa["seki"].values, alternative='two-sided'))

    #plt.yticks([0,1,2,3])
    #plt.xticks([-5,-3,-1])
    #plt.yscale("log")
    #plt.xscale("log")
    #plt.xticks([10**-6,10**-4,10**-2])
    plt.show()

